# Assignment 2 : Languague Modeling

# Recurrent Neural Networks and Language Models

You guys probably very excited about ChatGPT.  In today class, we will be implementing a very simple language model, which is basically what ChatGPT is, but with a simple LSTM.  You will be surprised that it is not so difficult at all.

Paper that we base on is *Regularizing and Optimizing LSTM Language Models*, https://arxiv.org/abs/1708.02182

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda:0


In [2]:
import torchtext, datasets, math
from tqdm.auto import tqdm
import torchdata
torch.__version__, torchdata.__version__, torchtext.__version__

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('2.0.1+cu117', '0.6.1', '0.15.2+cpu')

# Task 1. Dataset Acquisition 

## 1. Load data - Wiki Text

We will be using wikitext which contains a large corpus of text, perfect for language modeling task.  This time, we will use the `datasets` library from HuggingFace to load.

In [3]:
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
# https://huggingface.co/datasets/codeparrot/github-jupyter-code-to-text/tree/main dataset
#there are raw and preprocessed version; we used the raw one and preprocessed ourselves for fun

In [4]:
import datasets 
dataset = datasets.load_dataset("codeparrot/github-jupyter-code-to-text")
dataset

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 3851.52it/s]


Dataset parquet downloaded and prepared to /home/todsavadt/.cache/huggingface/datasets/codeparrot___parquet/codeparrot--github-jupyter-code-to-text-14ed47a6ba32f30c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 1236.71it/s]


DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 47452
    })
    test: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 11864
    })
})

## 2. Preprocessing

### Tokenizing

Simply tokenize the given text to tokens.

In [5]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['content'])}

tokenized_dataset = dataset.map(
    tokenize_data, 
    remove_columns=['repo_name', 'path', 'license', 'content'], 
    fn_kwargs={'tokenizer': tokenizer}
)

tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens'],
        num_rows: 47452
    })
    test: Dataset({
        features: ['tokens'],
        num_rows: 11864
    })
})

In [6]:
print(tokenized_dataset['train'][0]['tokens'])

['import', 'numpy', 'as', 'np', 'from', 'tensorflow', 'import', 'keras', 'from', 'tensorflow', '.', 'keras', 'import', 'layers', 'explanation', 'simple', 'mnist', 'convnet', 'author', 'fchollet<br>', 'date', 'created', '2015/06/19<br>', 'last', 'modified', '2020/04/21<br>', 'description', 'a', 'simple', 'convnet', 'that', 'achieves', '~99%', 'test', 'accuracy', 'on', 'mnist', '.', 'setup', 'end', 'of', 'explanation', '#', 'model', '/', 'data', 'parameters', 'num_classes', '=', '10', 'input_shape', '=', '(', '28', ',', '28', ',', '1', ')', '#', 'the', 'data', ',', 'split', 'between', 'train', 'and', 'test', 'sets', '(', 'x_train', ',', 'y_train', ')', ',', '(', 'x_test', ',', 'y_test', ')', '=', 'keras', '.', 'datasets', '.', 'mnist', '.', 'load_data', '(', ')', '#', 'scale', 'images', 'to', 'the', '[0', ',', '1]', 'range', 'x_train', '=', 'x_train', '.', 'astype', '(', 'float32', ')', '/', '255', 'x_test', '=', 'x_test', '.', 'astype', '(', 'float32', ')', '/', '255', '#', 'make', 'sur

### Numericalizing

We will tell torchtext to add any word that has occurred at least three times in the dataset to the vocabulary because otherwise it would be too big.  Also we shall make sure to add `unk` and `eos`.

In [7]:
vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_dataset['train']['tokens'], 
    min_freq=3,
    specials=['<unk>', '<eos>'],
    special_first=True
)

# vocab.insert_token('<unk>', 0)
# vocab.insert_token('<eos>', 1)
vocab.set_default_index(vocab['<unk>'])

In [8]:
print(len(vocab))

611138


In [9]:
print(vocab.get_itos()[:10])

['<unk>', '<eos>', '.', ',', ')', '(', "'", 'the', '=', 'of']


## 3. Prepare the batch loader

### Prepare data

Given "Chaky loves eating at AIT", and "I really love deep learning", and given batch size = 3, we will get three batches of data "Chaky loves eating at", "AIT `<eos>` I really", "love deep learning `<eos>`".  

In [10]:
def get_data(dataset, vocab, batch_size):
    data = []
    for example in tqdm(dataset):
        if example['tokens']:
            tokens = example['tokens'].append('<eos>')
            tokens = [vocab[token] for token in example['tokens']]
            data.extend(tokens)
    data = torch.LongTensor(data)
    num_batches = data.shape[0] // batch_size
    data = data[:num_batches * batch_size]
    data = data.view(batch_size, num_batches) #view vs. reshape (whether data is contiguous)
    return data #[batch size, seq len]

In [11]:
tokenized_dataset_train = tokenized_dataset['train'].select(range(1000))
tokenized_dataset_test = tokenized_dataset['test'].select(range(100))

In [12]:
batch_size = 32
train_data = get_data(tokenized_dataset_train, vocab, batch_size)
valid_data = get_data(tokenized_dataset_test, vocab, batch_size)
# test_data  = get_data(tokenized_dataset['test'], vocab, batch_size)

100%|██████████| 100/100 [00:00<00:00, 1107.97it/s]


In [13]:
train_data.shape, valid_data.shape #[batch_size, all the next length]

(torch.Size([32, 68772]), torch.Size([32, 6375]))

## 4. Modeling 

In [14]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers, dropout_rate):
        super().__init__()
        self.num_layers = num_layers
        self.hid_dim    = hid_dim
        self.emb_dim    = emb_dim
        
        self.embedding  = nn.Embedding(vocab_size, emb_dim)
        self.lstm       = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers, dropout=dropout_rate, batch_first=True)
        self.dropout    = nn.Dropout(dropout_rate)
        self.fc         = nn.Linear(hid_dim, vocab_size)
        
        self.init_weights()
    
    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hid_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_other)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.emb_dim,
                self.hid_dim).uniform_(-init_range_other, init_range_other) #We
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hid_dim,   
                self.hid_dim).uniform_(-init_range_other, init_range_other) #Wh
    
    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        cell   = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        return hidden, cell
        
    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach() #not to be used for gradient computation
        cell   = cell.detach()
        return hidden, cell
        
    def forward(self, src, hidden):
        #src: [batch_size, seq len]
        embedding = self.dropout(self.embedding(src)) #harry potter is
        #embedding: [batch-size, seq len, emb dim]
        output, hidden = self.lstm(embedding, hidden)
        #ouput: [batch size, seq len, hid dim]
        #hidden: [num_layers * direction, seq len, hid_dim]
        output = self.dropout(output)
        prediction =self.fc(output)
        #prediction: [batch_size, seq_len, vocab_size]
        return prediction, hidden

# Task 2. Model Training

## 5. Training 

Follows very basic procedure.  One note is that some of the sequences that will be fed to the model may involve parts from different sequences in the original dataset or be a subset of one (depending on the decoding length). For this reason we will reset the hidden state every epoch, this is like assuming that the next batch of sequences is probably always a follow up on the previous in the original dataset.

In [15]:
vocab_size = len(vocab)
emb_dim = 1024                # 400 in the paper
hid_dim = 1024                # 1150 in the paper
num_layers = 2                # 3 in the paper
dropout_rate = 0.65              
lr = 1e-3                     

In [16]:
model = LSTMLanguageModel(vocab_size, emb_dim, hid_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 1,269,015,362 trainable parameters


In [17]:
def get_batch(data, seq_len, idx):
    #this data is from get_data()
    #train_data.shape # [batch_size, number of batches....]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [20]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) #prevent gradient explosion - clip is basically 
        optimizer.step()
        epoch_loss += loss.detach().item() * seq_len
    return epoch_loss / num_batches

In [21]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.detach().item() * seq_len
    return epoch_loss / num_batches

In [20]:
n_epochs = 10
seq_len  = 50
clip    = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):
    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/best-val-auto.pt')
        
    print(f'\tepoch: {epoch+1}')
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	epoch: 1
	Train Perplexity: 1654.533
	Valid Perplexity: 1671.347


	epoch: 2
	Train Perplexity: 1426.690
	Valid Perplexity: 642.907


	epoch: 3
	Train Perplexity: 134.567
	Valid Perplexity: 154.949


	epoch: 4
	Train Perplexity: 72.070
	Valid Perplexity: 120.079


	epoch: 5
	Train Perplexity: 52.214
	Valid Perplexity: 103.758


	epoch: 6
	Train Perplexity: 41.170
	Valid Perplexity: 94.365


	epoch: 7
	Train Perplexity: 34.144
	Valid Perplexity: 87.731


	epoch: 8
	Train Perplexity: 29.301
	Valid Perplexity: 85.357


	epoch: 9
	Train Perplexity: 25.687
	Valid Perplexity: 86.338


KeyboardInterrupt: 

## 6. Testing

In [25]:
model.load_state_dict(torch.load('./models/best-val-auto.pt',  map_location=device))
# test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
# print(f'Test Perplexity: {math.exp(test_loss):.3f}')

## 7. Real-world inference

Here we take the prompt, tokenize, encode and feed it into the model to get the predictions.  We then apply softmax while specifying that we want the output due to the last word in the sequence which represents the prediction for the next word.  We divide the logits by a temperature value to alter the model’s confidence by adjusting the softmax probability distribution.

Once we have the Softmax distribution, we randomly sample it to make our prediction on the next word. If we get <unk> then we give that another try.  Once we get <eos> we stop predicting.
    
We decode the prediction back to strings last lines.

In [23]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [26]:
prompt = 'import matplotlib.pyplot'
max_seq_len = 50
seed = 0
            #superdiverse   more diverse
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0] 
#sample from this distribution higher probability will get more change
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
import matplotlib . pyplot as plt %matplotlib inline explanation load the data end of explanation # create a function to use the following data to visualize the data . # get a set of the data and the output column , which is to be able to be able to do this . #

0.7
import matplotlib . pyplot as plt plt . rcparams[ ' figure . figsize ' ] = ' nearest ' plt . rcparams[ ' figure . figsize ' ] = ' gray ' # from mne . stats import gridsearchcv from sklearn . grid_search import randomforestclassifier from sklearn . model_selection import train_test_split , query from

0.75
import matplotlib . pyplot as plt plt . rcparams[ ' figure . figsize ' ] = ' nearest ' plt . rcparams[ ' figure . figsize ' ] = ' gray ' # from mne . naive_bayes import aligninfo from mne . mplot3d import randomforestclassifier from mne . api import decisiontreeregressor from sklearn .

0.8
import matplotlib . pyplot as plt plt . rcparams[ ' figure . figsize ' ] = 5 # plt . xlabel ( ' head ( mm ) ' ) plt . title ( ' hi